In [ ]:
# imports
import sys
import os
import cv2
import numpy as np
import pickle
import torch
import time

In [ ]:
# Utils

# Creating directory
def create_dir(addr):
    if not os.path.exists(addr):
        os.mkdir(addr)

# Delete folder and its content
def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

# Addresses

# Data Address
data_address = "data"

# Raw data
raw_address = "../input/indian-birds/Birds_25"
raw_train = "../input/indian-birds/Birds_25/train"
raw_test = "../input/indian-birds/Birds_25/test"
raw_val = "../input/indian-birds/Birds_25/val"

# Processed data
processed_address = "data/processed"
cat_address = "data/processed/cat.pkl"
stat_address = "data/processed/stats.npz"

# Temp Address
temp_address = "temp"

# Results
result_address = "results/"
base_model_result = "results/base_model"

# Init Structure
create_dir(temp_address)
create_dir(data_address)
create_dir(processed_address)
create_dir(raw_address)
create_dir(raw_train)
create_dir(raw_test)
create_dir(raw_val)
create_dir(result_address)

# Constants
random_seed = 68
n = 2
r = 25
batch_size = 32

In [ ]:
# Statistical Calculation

# Loading statistical data
def load_stat(stat_addr):
    x_stats = np.load(stat_addr)
    x_mean = x_stats['mean']
    x_std = x_stats['std']
    return x_mean, x_std

# Saving statistical data
def save_stat(lin_sum, quad_sum, stat_addr, total_ct):
    x_mean = (lin_sum/total_ct).astype(np.float64)
    x_std = np.sqrt(quad_sum/total_ct - x_mean**2).astype(np.float64)
    np.savez_compressed(stat_addr, mean = x_mean, std = x_std)
    print("Saved statistical processed data")

# Partition data into smaller fragments and returns sum and quad sum
def moment_data(category, shape, address, stat_addr, mode):
    # Reading Data and Preprocessing it
    lin_sum, quad_sum = np.zeros(shape, dtype=np.float64), np.zeros(shape, dtype=np.float64)
    ct_list = []
    for cat in category:
        cat_path = os.path.join(address, cat)
        ct_list.append(len(os.listdir(cat_path)))
        
        # Checking if already present
        if os.path.exists(stat_addr) or mode==1:
            continue
        
        # Reading each image of particular category
        count = 0
        for img_name in sorted(os.listdir(cat_path)):
            img_path = os.path.join(cat_path, img_name)
            img = np.array(cv2.imread(img_path))
            lin_sum += img.astype(np.float64)
            quad_sum += np.square(img.astype(np.float64))
            count += 1
        
        # log
        print(f"Read {cat}\tindex: {category.index(cat)}\tnum: {count}")

    if mode == 0:
        print("Read Train Data")
    else:
        print("Read Test Data")

    return lin_sum, quad_sum, ct_list

# Reading data fragments and normalizing them
def normalize(addr, category, processed_x_addr, processed_y_addr, norm, overwrite):
    ct = 0
    for cat in category:
        cat_addr = os.path.join(addr, cat)
        for img_name in sorted(os.listdir(cat_addr)):
            # Save Address
            x_addr = os.path.join(processed_x_addr, f'{ct}.pt')
            y_addr = os.path.join(processed_y_addr, f'{ct}.pt')
            if not overwrite and os.path.exists(x_addr) and os.path.exists(y_addr):
                ct += 1
                continue
            
            # Reading image
            img_path = os.path.join(cat_addr, img_name)
            img = np.array(cv2.imread(img_path), dtype=np.float64)
            
            # Normalizing image
            norm_img = norm(img)
            x = torch.tensor(norm_img)
            
            # Getting y
            y = np.zeros((len(category),), dtype=np.float64)
            y[category.index(cat)] = 1
            y = torch.Tensor(y)
            
            # Saving image
            torch.save(x, x_addr)
            torch.save(y, y_addr)
            
            # Incrementing counter
            ct += 1
        
        print(f"Preprocessed\tindex: {category.index(cat)}\t{cat}")

# Preprocessed data
def preprocess(address, mode, overwrite = False, cat_addr = None, stat_addr = None, shape = (256, 256, 3)):
    '''
    mode = 0: Training Mode
    mode = 1: Testing Mode
    '''

    if mode not in [0, 1]:
        raise Exception("Not a Valid Mode")

    # Identifying Categories
    if mode == 0:
        category = sorted(os.listdir(address))
        
        # Saving categories
        if overwrite or not os.path.exists(cat_addr):
            with open(cat_addr, 'wb') as cat_file:
                pickle.dump(category, cat_file)
        print("Assigned Categories")
        
    elif mode == 1:
        # Reading Category array
        with open(cat_addr, 'rb') as cat_file:
            category = pickle.load(cat_file)
        print("Read Categories")
    
    # count of images and their moments
    lin_sum, quad_sum, ct_list = moment_data(category=category, shape=shape, address=address, stat_addr=stat_addr, mode=mode)
        
    # Normalizing Data
    if mode == 0 and (overwrite or not os.path.exists(stat_addr)):
        save_stat(lin_sum=lin_sum, quad_sum=quad_sum, stat_addr=stat_addr, total_ct=sum(ct_list))
    x_mean, x_std = load_stat(stat_addr)
        
    # # Function for normalization
    # norm = lambda img: np.divide((img - x_mean), x_std, out = np.zeros_like(x_mean), where = x_std!=0)

    # # Creating train directory
    # create_dir(processed_x_addr)
    # create_dir(processed_y_addr)

    # # Reading Saved Files and Normalizing them
    # normalize(addr=address, category=category, processed_x_addr=processed_x_addr, processed_y_addr=processed_y_addr, norm=norm, overwrite=overwrite)

    return category

# Function Call
category = preprocess(address=raw_train, mode=0, overwrite=False, cat_addr=cat_address, stat_addr=stat_address)

In [ ]:
# cuda
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Working with {device}")

In [ ]:
# Data Loader
def generate_cat_list(address):
    cat_list = []
    for cat in category:
        cat_path = os.path.join(address, cat)
        cat_list.append(sorted(os.listdir(cat_path)))
    return cat_list

# Data Generator for Training Data
def data_generator(ind, address, cat_list, device):
    for cat_ind in range(len(cat_list)):
        if ind < len(cat_list[cat_ind]):
            cat_path = os.path.join(address, category[cat_ind])
            img_path = os.path.join(cat_path, cat_list[cat_ind][ind])
            x = torch.tensor(cv2.imread(img_path), dtype=torch.float64)
            y_vec = np.zeros((len(category),))
            y_vec[cat_ind] = 1
            y = torch.tensor(y_vec, dtype=torch.float64)
            return x, y
        else:
            ind -= len(cat_list[cat_ind])

# Data Loader
class DataLoader():
    def __init__(self, address, cat_list, batch_size, stat_addr, rand_seed = None, randomize = True, device = device):
        # Finding number of Data Samples
        self.num_data = sum([len(cat_arr) for cat_arr in cat_list])
        
        # Randomizing if True
        np.random.seed(rand_seed)
        self.order = np.arange(self.num_data)
        
        # Assigning Class Variables
        self.batch_size = batch_size
        self.device = device
        self.randomize = randomize
        
        # Load Statsistical data
        self.mean, self.std = load_stat(stat_addr)
        self.mean = torch.tensor(self.mean).to(device)
        self.std = torch.tensor(self.std).to(device)

        # Data Generator Function
        self.generate = lambda ind: data_generator(ind, address, cat_list, device = device)
        
    def __iter__(self):
        # Initializing index
        self.ind = 0
        if self.randomize:
            np.random.shuffle(self.order)
        return self
    
    def __next__(self):
        # Checking stop condition
        if self.ind >= self.num_data:
            raise StopIteration
        
        X, Y = [], []
        for i in range(self.batch_size):
            # Loading X, Y
            x, y = self.generate(self.order[self.ind])
            X.append(x.permute(2, 0, 1))    # Channel is first dimension
            Y.append(y)
            
            # Updating index
            self.ind += 1
            if self.ind == self.num_data:
                break
        
        # Returning data
        X = torch.stack(X).to(self.device)
        Y = torch.stack(Y).to(self.device)
        return X, Y

# Creating Data Loader
data_loader_train = DataLoader(address=raw_train,
                               cat_list=generate_cat_list(raw_train),
                               batch_size=batch_size,
                               stat_addr=stat_address,
                               rand_seed=random_seed)

data_loader_test = DataLoader(address=raw_test,
                               cat_list=generate_cat_list(raw_test),
                               batch_size=batch_size,
                               stat_addr=stat_address,
                               rand_seed=random_seed)

data_loader_val = DataLoader(address=raw_val,
                               cat_list=generate_cat_list(raw_val),
                               batch_size=batch_size,
                               stat_addr=stat_address,
                               rand_seed=random_seed)


In [ ]:
# Model

class ResBlock(torch.nn.Module):
    def __init__(self, in_channel, out_channel, norm_func, kernel_size=3, stride=1):
        super(ResBlock, self).__init__()
        
        self.conv1 = torch.nn.Conv2d(in_channel, in_channel, kernel_size, stride=1, padding=1, dtype=torch.float64)
        self.norm1 = norm_func(in_channel, dtype=torch.float64)
        self.activation1 = torch.nn.ReLU()
        
        self.conv2 = torch.nn.Conv2d(in_channel, out_channel, kernel_size, stride, padding=1, dtype=torch.float64)
        self.norm2 = norm_func(out_channel, dtype=torch.float64)
        self.activation2 = torch.nn.ReLU()
        
        self.project = True if (in_channel != out_channel) or (stride != 1) else False
        if self.project:
            self.conv_project = torch.nn.Conv2d(in_channel, out_channel, kernel_size, stride, padding=1, dtype=torch.float64)

    def forward(self, x):
        res = x
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.activation1(x)
        
        x = self.conv2(x)
        x = self.norm2(x)
        x += self.conv_project(res) if self.project else res
        x = self.activation2(x)
        
        return x

class Resnet(torch.nn.Module):
    def __init__(self, n, r, norm_func = torch.nn.BatchNorm2d):
        super(Resnet, self).__init__()

        self.norm = norm_func
        
        #Input
        self.input_layer = []
        self.input_layer.append(torch.nn.Conv2d(3, 16, 3, 1, padding=1, dtype=torch.float64))
        self.input_layer.append(self.norm(16, dtype=torch.float64))
        self.input_layer.append(torch.nn.ReLU())
        self.input_layer = torch.nn.Sequential(*self.input_layer)
        
        # Layer1
        self.hidden_layer1 = []
        for i in range(n):
            self.hidden_layer1.append(ResBlock(16, 16, self.norm))
        self.hidden_layer1 = torch.nn.Sequential(*self.hidden_layer1)
        
        # Layer2
        self.hidden_layer2 = []
        self.hidden_layer2.append(ResBlock(16, 32, self.norm, stride = 2))
        for i in range(n-1):
            self.hidden_layer2.append(ResBlock(32, 32, self.norm))
        self.hidden_layer2 = torch.nn.Sequential(*self.hidden_layer2)
        
        # Layer3
        self.hidden_layer3 = []
        self.hidden_layer3.append(ResBlock(32, 64, self.norm, stride = 2))
        for i in range(n-1):
            self.hidden_layer3.append(ResBlock(64, 64, self.norm))
        self.hidden_layer3 = torch.nn.Sequential(*self.hidden_layer3)
            
        # Pool Layer
        self.pool = torch.nn.AdaptiveAvgPool2d(1)
        self.flatten = torch.nn.Flatten()
        
        # Output Layer
        self.output_layer = []
        self.output_layer.append(torch.nn.Linear(64, r, dtype=torch.float64))
        self.output_layer.append(torch.nn.Softmax(1))
        self.output_layer = torch.nn.Sequential(*self.output_layer)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.hidden_layer1(x)
        x = self.hidden_layer2(x)
        x = self.hidden_layer3(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.output_layer(x)
        
        return x

# Setting Random Seed
torch.manual_seed(random_seed)
resnet_base_model = Resnet(n, r).to(device)

In [ ]:
# Training Model

def train(model, data_loader, save_addr, num_epoch = 50, learning_rate = 1e-4, overwrite = False):
    # Creating save folder
    create_dir(save_addr)
    model_addr = os.path.join(save_addr, 'model')
    create_dir(model_addr)
    loss_addr = os.path.join(save_addr, 'loss')
    create_dir(loss_addr)

    # Parameters for training
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    start_time = time.time()
    
    for epoch in range(num_epoch):
        batch_ct = 0
        epoch_loss = 0

        # Loading previous model
        epoch_addr = os.path.join(model_addr, f'{epoch}.pt')
        epoch_loss_addr = os.path.join(loss_addr, f'{epoch}.pt')

        if not overwrite and os.path.exists(epoch_addr) and os.path.exists(epoch_loss_addr):
            model.load_state_dict(torch.load(epoch_addr))
            epoch_loss = torch.load(epoch_loss_addr)

            print(f"Epoch: {epoch} Loaded\t\tLoss: {epoch_loss}")
        else:
            # Training next epoch
            for x, y in data_loader:
                y_pred = model(x)
                loss = loss_fn(y_pred, y)
                epoch_loss += loss.item()
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                batch_ct += 1
                print(f"\tBatch: {batch_ct}\tLoss: {round(loss.item(), 6)}\tTotal Loss: {round(epoch_loss/batch_ct, 6)}\tTime: {time.time()-start_time}")

            # Saving model after each epoch
            torch.save(model.state_dict(), epoch_addr)
            torch.save(epoch_loss/batch_ct, epoch_loss_addr)

            print(f"Epoch: {epoch}\tLoss: {round(epoch_loss/batch_ct, 6)}\tTime: {time.time() - start_time}")

train(resnet_base_model, data_loader_train, base_model_result)